In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
plt.style.use('seaborn')

import yfinance as yf



C:\Users\Himanshu Gupta\AppData\Local\Temp\ipykernel_19568\2010723511.py:5: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


In [2]:
indices=['^GSPC','^IXIC','^FTSE','^DJI','^GDAXI']
equities = ['AMZW', 'SONY', 'GOOG', 'META', 'NVDA']

In [3]:
start_date='2010-01-01'
end_date='2023-05-01'

indices_data = yf.download(indices, start=start_date, end=end_date)['Adj Close']


[*********************100%***********************]  5 of 5 completed


In [4]:

equities_data = yf.download(equities, start=start_date, end=end_date)['Adj Close']

[*********************100%***********************]  5 of 5 completed

1 Failed download:
- AMZW: No timezone found, symbol may be delisted


In [5]:
indices_returns = indices_data.pct_change()
equities_returns = equities_data.pct_change()


In [6]:
indices_cumulative_returns = (1 + indices_returns).cumprod()
equities_cumulative_returns = (1 + equities_returns).cumprod()


In [7]:
print(indices_cumulative_returns)


                ^DJI     ^FTSE    ^GDAXI     ^GSPC     ^IXIC
Date                                                        
2010-01-04       NaN       NaN       NaN       NaN       NaN
2010-01-05  0.998872  1.004036  0.997282  1.003116  1.000126
2010-01-06  0.999029  1.005400  0.997690  1.003663  0.996825
2010-01-07  1.002164  1.004800  0.995215  1.007679  0.996374
2010-01-08  1.003234  1.006163  0.998233  1.010583  1.003790
...              ...       ...       ...       ...       ...
2023-04-24  3.200636  1.438503  2.622878  3.651436  5.214476
2023-04-25  3.168080  1.434667  2.624230  3.593703  5.111358
2023-04-26  3.146447  1.427668  2.611598  3.579899  5.135266
2023-04-27  3.195983  1.423850  2.612379  3.649944  5.259979
2023-04-28  3.221683  1.430940  2.632538  3.680068  5.296515

[3441 rows x 5 columns]


In [8]:
print(equities_cumulative_returns)

            AMZW      GOOG      META       NVDA      SONY
Date                                                     
2010-01-04   NaN       NaN       NaN        NaN       NaN
2010-01-05   NaN  0.995596       NaN   1.014603  0.995336
2010-01-06   NaN  0.970499       NaN   1.021092  0.994337
2010-01-07   NaN  0.947906       NaN   1.001081  0.992672
2010-01-08   NaN  0.960542       NaN   1.003245  1.012991
...          ...       ...       ...        ...       ...
2023-04-24   NaN  6.840382  5.566047  63.754404  3.049967
2023-04-25   NaN  6.701371  5.428983  61.865959  3.023318
2023-04-26   NaN  6.691121  5.477374  63.551646  3.124917
2023-04-27   NaN  6.942239  6.240126  64.188203  3.167555
2023-04-28   NaN  6.932629  6.286163  65.421226  2.987009

[3353 rows x 5 columns]


In [9]:
def calculate_max_drawdown(data):
    cumulative_returns = (1 + data).cumprod()
    peak = cumulative_returns.expanding(min_periods=1).max()
    drawdown = (cumulative_returns / peak) - 1
    max_drawdown = drawdown.min()
    return max_drawdown

indices_max_drawdowns = calculate_max_drawdown(indices_returns)
equities_max_drawdowns = calculate_max_drawdown(equities_returns)


In [10]:
print(indices_max_drawdowns)
print(equities_max_drawdowns)

^DJI     -0.370862
^FTSE    -0.366055
^GDAXI   -0.387794
^GSPC    -0.339250
^IXIC    -0.363953
dtype: float64
AMZW         NaN
GOOG   -0.446018
META   -0.767361
NVDA   -0.663351
SONY   -0.761869
dtype: float64


In [11]:
risk_free_rate = 0.0  # Set the risk-free rate

def calculate_sharpe_ratio(data, risk_free_rate):
    excess_returns = data - risk_free_rate
    annualized_returns = excess_returns.mean() * 252  # Assuming 252 trading days in a year
    annualized_volatility = data.std() * np.sqrt(252)
    sharpe_ratio = annualized_returns / annualized_volatility
    return sharpe_ratio

def calculate_sortino_ratio(data, risk_free_rate):
    downside_returns = data.copy()
    downside_returns[downside_returns > 0] = 0  # Consider only negative returns for downside
    annualized_returns = data.mean() * 252
    downside_deviation = downside_returns.std() * np.sqrt(252)
    sortino_ratio = (annualized_returns - risk_free_rate) / downside_deviation
    return sortino_ratio

indices_sharpe_ratio = calculate_sharpe_ratio(indices_returns, risk_free_rate)
indices_sortino_ratio = calculate_sortino_ratio(indices_returns,risk_free_rate)


In [12]:
equities_sharpe_ratio=calculate_sharpe_ratio(equities_returns,risk_free_rate)
equities_sortino_ratio=calculate_sortino_ratio(equities_returns,risk_free_rate)

In [13]:
print(equities_sharpe_ratio)
print(equities_sortino_ratio)

AMZW         NaN
GOOG    0.668684
META    0.616949
NVDA    0.924979
SONY    0.415726
dtype: float64
AMZW         NaN
GOOG    1.121063
META    1.013448
NVDA    1.607864
SONY    0.717438
dtype: float64


In [14]:
print(indices_sharpe_ratio)
print(indices_sortino_ratio)

^DJI      0.590550
^FTSE     0.244006
^GDAXI    0.453854
^GSPC     0.632504
^IXIC     0.702592
dtype: float64
^DJI      0.904982
^FTSE     0.378797
^GDAXI    0.716547
^GSPC     0.974461
^IXIC     1.095676
dtype: float64
